![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_InstructorEmbeddings.ipynb)

# Import ONNX InstructorEmbeddings  models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in `Spark NLP 5.1.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.


## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.48.3 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to export ONNX models directly via the CLI with `!optimum-cli export onnx --model MODEL_NAME EXPORT_PATH`.
- We'll use the [hkunlp/instructor-base](https://huggingface.co/hkunlp/instructor-base) model from HuggingFace as an example and export it with the `optimum-cli`.

In [2]:
MODEL_NAME = "hkunlp/instructor-base"
EXPORT_PATH = f"export_onnx/{MODEL_NAME}"

! optimum-cli export onnx --model {MODEL_NAME} {EXPORT_PATH} --task feature-extraction

2025-06-14 03:44:41.159023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749872681.505882     983 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749872681.602460     983 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-14 03:44:42.349689: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
modules.json: 100% 461/461 [00:00<00:00, 2.79MB/s]
config_sentence_transformers.json: 100% 122/122 [00:00<00:00, 387kB/s]
REA

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 433936
-rw-r--r-- 1 root root      1585 Jun 14 03:45 config.json
-rw-r--r-- 1 root root 441093862 Jun 14 03:45 model.onnx
-rw-r--r-- 1 root root      2543 Jun 14 03:45 special_tokens_map.json
-rw-r--r-- 1 root root    791656 Jun 14 03:45 spiece.model
-rw-r--r-- 1 root root     20996 Jun 14 03:45 tokenizer_config.json
-rw-r--r-- 1 root root   2422434 Jun 14 03:45 tokenizer.json


Spark NLP requires auxiliary model files, such as the model file, to be placed in a dedicated `assets` directory for proper loading and compatibility.

In [4]:
!mkdir -p {EXPORT_PATH}/assets && mv -t {EXPORT_PATH}/assets  {EXPORT_PATH}/*.model

In [5]:
!ls -l {EXPORT_PATH}/assets

total 776
-rw-r--r-- 1 root root 791656 Jun 14 03:45 spiece.model


## Import and Save InstructorEmbeddings  in Spark NLP

- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 33.3 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `InstructorEmbeddings ` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `InstructorEmbeddings ` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [8]:
from sparknlp.annotator import InstructorEmbeddings

embedding = InstructorEmbeddings.loadSavedModel(
      EXPORT_PATH,
      spark
      )\
    .setInputCols(["document"])\
    .setOutputCol("instructor")

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
embedding.write().overwrite().save("./{}_spark_nlp".format(EXPORT_PATH))

Awesome  😎 !

This is your ONNX InstructorEmbeddings  model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [10]:
! ls -l {EXPORT_PATH}_spark_nlp

total 431608
-rw-r--r-- 1 root root 441161306 Jun 14 03:49 instructor_onnx
-rw-r--r-- 1 root root    791656 Jun 14 03:49 instructor_spp
drwxr-xr-x 2 root root      4096 Jun 14 03:49 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny InstructorEmbeddings  model 😊

In [11]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

instructor_loaded = InstructorEmbeddings.load(f"{EXPORT_PATH}_spark_nlp")\
    .setInputCols(["document"])\
    .setOutputCol("instructor")\
    .setInstruction("Encode This:")

pipeline = Pipeline(stages=[
    document_assembler,
    instructor_loaded
])

data = spark.createDataFrame([[
    "William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist."
]]).toDF("text")

model = pipeline.fit(data)
result = model.transform(data)

result.select("instructor.result").show()

+--------------------+
|              result|
+--------------------+
|[William Henry Ga...|
+--------------------+



That's it! You can now go wild and use hundreds of InstructorEmbeddings  models from HuggingFace 🤗 in Spark NLP 🚀
